# Autodoc scraper

This notebook is needed to reliably download the following vehicle metadata: 
- Make
- Model
- Chassiscode
- Production year

An example to illustrate the problem: Ford Escort.
Most people reference to a car like this it's a 'Ford Escort', an 'Audi TT', a Volkswagen Golf... whatever. This is problematic for two main reasons:
1) Generations:
    The Ford Escort MK1 was introduced in 1967 and ran until 1975 after which it was followed by the mk2 (1974-1980), the mk3(1980-1986), the mk4(1986-1990/1992). Each generation had their own distinct styling, with MK1 and MK2 being separate design, the MK4-generation is generally concidered as a facelift of the MK3 with new engine options and slight improvements in the gearbox design.
   For image recognition purposes we'd like to have metadata of advertisements (autoscout...) that include information to make a guesstimate of the car's generation (i.e. Date of first registration - problematic for various reasons, but we have no better option).
   
2) Chassis Types:
    The MK4 has a variety of body styles - these codes are typically easy, convenenient ways to identify specific shell types so that parts manufacturers can easily tell which cars a part fits to. Going back to the MK4 Ford Escort, the following shells were available in Europe:


   	- 3/5-door liftback
    - 3/5-door estate
    - 2-door cabriolet (ALF)
    - 3-door van


By collecting a proper database, we can generate target data for an ML model that can more accurately determine a Vehicle. 

In [1]:
import json
import pandas as pd
import cloudscraper
import re
import time
import random
from tqdm import tqdm

In [2]:
url = 'https://www.autodoc.be'
scraper = cloudscraper.create_scraper(
    browser={
        'browser': 'firefox',
        'platform': 'windows',
        'mobile': False
    },
    delay = 5)  # returns a CloudScraper instance
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5', 
    'Referrer': 'https://www.google.com/search?q=autodoc'
}
scraper.get(url, headers=headers).content  #we don't need the reply from this, just need to set up a connection with cloudscraper. 
;

''

In [3]:
brands_url = 'https://www.autodoc.be/ajax/selector/vehicle'
#get brands:
brands = scraper.get(brands_url, headers=headers).json()

In [4]:
brandsdf = pd.DataFrame(brands['makers'][1]['options'])

In [5]:
def get_brand_models(make_id):
    model_url = f'https://www.autodoc.be/ajax/selector/vehicle?maker={make_id}&action=models'
    models = scraper.get(model_url, headers=headers).json()
    return models
#change brand to whatever you want to experiment with the code and parser. 
brand = "volkswagen"
r = brandsdf[brandsdf['name'].str.fullmatch(brand, na=False, case=False)]
brand_id = r.id.reset_index().id[0]
models = get_brand_models(brand_id)

In [6]:
def parsename(name):
    """not all manufacturers use chassis codes (i.e. Ferrari), some use chassis codes,
    but only for some models (i.e. Maserati). We always assume that the final () is the 
    production date, the beginning is the modelname - with some indication about chassis
    layout. IF there's a middle (), it's the chassiscode.
    Year data is notated: MM.YYYY - MM.YYYY with the second date optionally being ... for
    current production runs. """
    # Regex to find all parentheses content
    parts = re.findall(r'\(([^)]+)\)', name)

    
    # Extract model name (text before first parentheses, if any)
    model_name = name.split('(')[0].strip()
    
    # Initialize chassis_code, start_date, end_date as None
    chassis_code, start_date, end_date = None, None, None
    
    # Process parts found in parentheses
    if len(parts) == 1:
        # Only one set of parentheses, assume it's the production date
        production = parts[0]
    elif len(parts) == 2:
        # Two sets of parentheses, assume the first is chassis code, second is production date
        chassis_code, production = parts
    else:
        # If there's an unexpected format, return what we can safely parse
        return model_name, chassis_code, start_date, end_date
    
    # Parse production dates
    if production:
        # Regex to match dates in format MM.YYYY - MM.YYYY or MM.YYYY - ....
        date_match = re.match(r'(\d{2}\.\d{4}) - (\d{2}\.\d{4}|\.{3})', production)
        if date_match:
            start_date = date_match.group(1)
            end_date = date_match.group(2) if date_match.group(2) != "..." else None
    
    return model_name, chassis_code, start_date, end_date

result = []
for key in models['models']:
    seriesname = key['label']
    submodels = key['options']
    for submodel in submodels:
        submodelid = submodel['id']
        submodelname = submodel['name']
        name, codes, start, end = parsename(submodelname)
        result.append([brand, seriesname, submodelid, submodelname, name, codes, start, end])
        
    #full_name = seriescodes['name']
    #print( seriesname, full_name)
    #name, codes, start, end = parsename(full_name)

In [7]:
r = pd.DataFrame(result)

In [8]:
r.head(10)


,0,1,2,3,4,5,6,7
0,volkswagen,166,4644,166 SUV Cabrio (02.1942 - 12.1945),166 SUV Cabrio,None,02.1942,12.1945
1,volkswagen,181,512,Safari (09.1969 - 11.1983),Safari,None,09.1969,11.1983
2,volkswagen,411/412,1616,411/412 Hatchback (4) (07.1968 - 03.1975),411/412 Hatchback,4,07.1968,03.1975
3,volkswagen,411/412,405,412 Variant (09.1969 - 03.1975),412 Variant,None,09.1969,03.1975
4,volkswagen,1500/1600,1620,1500/1600 Hatchback (31) (08.1965 - 03.1975),1500/1600 Hatchback,31,08.1965,03.1975
5,volkswagen,1500/1600,507,1500/1600 Sedan (31) (04.1961 - 12.1973),1500/1600 Sedan,31,04.1961,12.1973
6,volkswagen,1500/1600,1621,1500/1600 Variant (36) (04.1961 - 07.1973),1500/1600 Variant,36,04.1961,07.1973
7,volkswagen,AMAROK,14172,"Amarok Flatbed Vrachtwagen / Chassis (S1B, S6B...",Amarok Flatbed Vrachtwagen / Chassis,"S1B, S6B, S7B",09.2011,None
8,volkswagen,AMAROK,8636,"Amarok Pick-up (2HA, 2HB, S1B, S6B, S7A, S7B) ...",Amarok Pick-up,"2HA, 2HB, S1B, S6B, S7A, S7B",09.2010,None
9,volkswagen,AMEO,36973,Ameo (631) (05.2016 - ...),Ameo,631,05.2016,None


Code above was to test for different manufacturers, models and scenarios. The code should be good enough to parse the whole Autodoc database with. We'll use a for loop and the functions written above to build our own vehicle database. 

In [11]:
all_results = []
for row in tqdm(brandsdf.iterrows()):
    dodotime = random.randint(5, 17)
    make_id = row[1]['id']
    brand_name = row[1]['name']
    models = get_brand_models(make_id)
    for key in models['models']:
        seriesname = key['label']
        submodels = key['options']
        for submodel in submodels:
            submodelid = submodel['id']
            submodelname = submodel['name']
            name, codes, start, end = parsename(submodelname)
            all_results.append([brand_name, seriesname, submodelid, submodelname, name, codes, start, end])
    time.sleep(dodotime)#mimick human browsing by having a sleeptime in between requests with random intervals

24it [04:04,  8.70s/it]

12.2011 - ...


104it [18:50, 10.29s/it]

06.1996 - 03.2003
03.2003 - 07.2009


172it [31:54, 11.13s/it]


In [98]:
all_models = pd.DataFrame(all_results)
columns = ['brand', 'series', 'submodel_id', 'submodel_name', 'modelname', 'chassiscode', 'start_production', 'end_production']
all_models.columns = columns
all_models.sample(5)


,brand,series,submodel_id,submodel_name,modelname,chassiscode,start_production,end_production
1296,FERRARI,360,4182,360 Modena (03.1999 - 03.2005),360 Modena,None,03.1999,03.2005
1310,FERRARI,550 / 575M Maranello,1569,550 / 575M Maranello (F133) (04.1996 - 12.2006),550 / 575M Maranello,F133,04.1996,12.2006
558,BRABUS,40S,13085,40S (W221) (04.2011 - ...),40S,W221,04.2011,None
4243,PORSCHE,356,1870,356 Coupe (09.1947 - 03.1968),356 Coupe,None,09.1947,03.1968
4032,PEUGEOT,106,1559,"106 II Hatchback (1A, 1C) (04.1996 - 05.2005)",106 II Hatchback,"1A, 1C",04.1996,05.2005


In [99]:
all_models['doorcount'] = -1
all_models['chassis_style'] = ''
all_models.sample(5)

,brand,series,submodel_id,submodel_name,modelname,chassiscode,start_production,end_production,doorcount,chassis_style
3777,NISSAN,TRADE,7617,Trade Transporter (06.1996 - 10.2001),Trade Transporter,None,06.1996,10.2001,-1,
1295,FERRARI,348,2083,348 ts/GTS Targa (09.1988 - 12.1995),348 ts/GTS Targa,None,09.1988,12.1995,-1,
1877,FORD USA,RANGER,12004,Ranger Stripped Chassis (09.1997 - 12.2012),Ranger Stripped Chassis,None,09.1997,12.2012,-1,
3521,MOSKVICH,403,1852,403 Sedan (03.1962 - 09.1968),403 Sedan,None,03.1962,09.1968,-1,
5349,TRABANT,P 601,1706,P 601 Bestelwagen (07.1966 - 04.1990),P 601 Bestelwagen,None,07.1966,04.1990,-1,


We have some rows with multiple chassis codes in a single cell. These are often facelift versions and need a special way to fix it. The first entry is for isntance the Alfa Romeo GTV - 916, 916C where C indicates a facelift version without much underlying mechanical work. It should be noted that the production years should be split up too. We can use Gemini for this in a later phase. We need Gemini anyway to find a solution for not-found chassis-styles and missing doorcount values. 

In [100]:
#mark rows with 'reparsed == 1' if the original chassiscode contained a ',';
all_models['reparsed'] = all_models['chassiscode'].str.contains(',', case=False, na=False).astype(int)
#filter out these rows and start to split them up. 
comma_rows = all_models.loc[all_models['reparsed'] == 1]
#split comma_rows per row on chassistyle (,), and make mutliple new rows per comma separated value:
expanded_rows = []
for index, row in comma_rows.iterrows():
    chassis_codes = row['chassiscode'].split(',')  # Split by comma
    for code in chassis_codes:
        new_row = row.copy()  # Create a copy of the original row
        new_row['chassiscode'] = code.strip()  # Assign the split value (remove any leading/trailing spaces)
        expanded_rows.append(new_row)
expanded_df = pd.DataFrame(expanded_rows)
#expanded_df.head(5)  ##GOOD!
#get non-comma rows
non_comma_rows = all_models.loc[all_models['reparsed'] != 1]
all_models_final = pd.concat([non_comma_rows, expanded_df], ignore_index=True)


Now we'll try to to extract chassis styles from the submodel_name.HTere are some giveaways such as cabrio, convertible, sedan, coupe, hatch, hatchback.... We do this to make a more uniform chassis_style selection possible.

In [108]:
#we can easily convert at least some well known car name elements to the body styles used by autoscout.
#we'll be using autoscout later to scrape trainingdata. Autoscout offers a value for bodyshape, which 
# can be used to improve quality of feature data. It's not always very well defined, e.g. Hyundai I20
# has entries listed as a 'Berline' and a 'Stadswagen' for same chassisstyles. This is a caveat to be 
# wary of later.
conversion_dict = {
    'Cabriolet': ['convertible', 'cabrio', 'cabriolet', 'roadster', 'spyder', 'spider', 'soft-top', 'targa'],  ## e.g.: Alfa Romeo Spider
    'Break': ['stationwagon', 'wagon', 'estate', 'break', 'combi'],  ## e.g. Volvo (V-XX series - V-40, V-50, V-60....)
    'SUV': ['suv', 'crossover', 'jeep', 'Pick-up', 'flatbed'],       ## e.g. (combines pickup and SUV in here) Hyundai Tucson, Mitsubishi L200
    'Berline': ['sedan', 'saloon'],  ## e.g. Volvo S-XX series: S-40, S-60....
    'Monovolume': ['minivan', 'mpv'], 
    'Coupe': ['coupe', 'coupé'],   #e.g. BMW 2series coupe (not the 2 series tourer)
    'Stadswagen': ['hatchback', 'hatch'],  #problematic category on autoscout, we'll roll with it anyway
}
for k,v in conversion_dict.items():
    v = '|'.join(v)
    all_models_final.loc[all_models_final['submodel_name'].str.contains(rf'{v}', case=False, na=False), 'chassis_style'] = k


In [109]:
## how many chassis styles are still empty?
all_models_final[all_models_final['chassis_style'] == ''].shape

(2839, 11)

In [214]:
#there are some cars where we know that they have X doors whatsoever. Porsche 911 was always a 2 door
# Volvo V-series were always 5 door cars... by implementing domain knowledge, we can reduce the need for 
# further calls to gemini or other datasets. (actuallly gemini is pretty useless for this, it makes too much mistakes)
known_layouts = {
    2: {
        'PORSCHE': ['911', '718', '356', '912', '914', '918', '924', '928', '944', 'CAYMAN', '959', '968', 'BOXSTER', 'CARRERA GT'],
        'TOYOTA': ['MR2', 'CELICA'], 
        'FIAT': ['X 1/9', '124'], 
        'PEUGEOT': ['RCZ'], 
        'ABARTH': ['124'],
        'ALFA ROMEO': ['SPIDER'], 
        'AUDI': ['TT', 'R8']
    }, 
    3: {
        'VOLVO': ['C30'], 
        'ALFA ROMEO': ['MITO']
    },
    5: {
        'PORSCHE': ['PANAMERA', 'MACAN', 'CAYENNE'], 
        'VOLVO': ['V40','V50', 'V60', 'V70', 'V90', 'XC40', 'XC60', 'XC90','S40', 'S70', 'S80', 'XC70'], 
        'VOLKSWAGEN': ['PHAETON', 'TIGUAN', 'TOUAREG', 'TOURAN','PASSAT'], 
        'PEUGEOT': ['5008', '4008'], 
        'FIAT': ['MULTIPLA'], 
        'ALFA ROMEO': ['STELVIO']
    }
}
known_styles = {
    "SUV": {
        'VOLVO': ['XC90']
    }
}


In [215]:
for doorcount, cars in known_layouts.items():
    for brand, models in cars.items():
        for model in models:
            all_models_final.loc[(all_models_final['brand'] == brand) & (all_models_final['series'] == model), 'doorcount'] = doorcount

for chassistype, cars in known_styles.items():
    for brand, models in cars.items():
        for model in models:
            all_models_final.loc[(all_models_final['brand'] == brand) & (all_models_final['series'] == model), 'chassis_style'] = chassistype
    

In [216]:
all_models_final.query('brand=="ALFA ROMEO" and doorcount==-1')['series'].unique()

array(['4C', '6', '8C', '33', '75', '90', '145', '146', '147', '155',
       '156', '159', '164', '166', '1750-2000', '1900', '2600', 'ALFASUD',
       'ALFETTA', 'AR', 'ARNA', 'BERLINA', 'BRERA', 'DISCO', 'GIULIA',
       'GIULIETTA', 'GT', 'GTA', 'GTV', 'MONTREAL', 'RZ', 'SZ', 'TONALE'],
      dtype=object)

In [217]:
# Update the 'doorcount' column to x where 'brand' is y and 'series' is z
print(
    all_models_final[all_models_final['doorcount'] == -1].shape[0], 
    all_models_final[all_models_final['doorcount'] != -1].shape[0],
    all_models_final[(all_models_final['doorcount'] == -1) | (all_models_final['chassis_style'] != '')].shape[0]

)


6457 198 6559


In [218]:
all_models_final.shape

(6655, 11)

In [221]:
brand = 'BMW'
model = '1 SERIES'
all_models_final.loc[(all_models_final['brand'] == brand) & (all_models_final['series'] == model)]

,brand,series,submodel_id,submodel_name,modelname,chassiscode,start_production,end_production,doorcount,chassis_style,reparsed


In [220]:
all_models_final

,brand,series,submodel_id,submodel_name,modelname,chassiscode,start_production,end_production,doorcount,chassis_style,reparsed
0,ABARTH,124,36454,124 Spider (348) (03.2016 - ...),124 Spider,348,03.2016,None,2,Cabriolet,0
1,ABARTH,500 / 595 / 695,14669,500 / 595 / 695 Hatchback (312) (01.2008 - ...),500 / 595 / 695 Hatchback,312,01.2008,None,-1,Stadswagen,0
2,ABARTH,500 / 595 / 695,14670,500C / 595C / 695C Cabrio (312) (09.2009 - ...),500C / 595C / 695C Cabrio,312,09.2009,None,-1,Cabriolet,0
3,ABARTH,GRANDE PUNTO,14671,Grande Punto (199_) (12.2007 - 12.2012),Grande Punto,199_,12.2007,12.2012,-1,,0
4,ABARTH,PUNTO,14673,Punto (199) (03.2012 - ...),Punto,199,03.2012,None,-1,,0
...,...,...,...,...,...,...,...,...,...,...,...
6650,VOLVO,XC70,4867,"XC70 I Cross Country (P2, 295) (10.1997 - 10.2...",XC70 I Cross Country,295,10.1997,10.2007,5,,1
6651,VOLVO,XC70,6391,"XC70 II Station Wagon (P24, 136) (04.2007 - 12...",XC70 II Station Wagon,P24,04.2007,12.2016,5,Break,1
6652,VOLVO,XC70,6391,"XC70 II Station Wagon (P24, 136) (04.2007 - 12...",XC70 II Station Wagon,136,04.2007,12.2016,5,Break,1
6653,VOLVO,XC90,4879,"XC90 I (C, 275) (06.2002 - 12.2014)",XC90 I,C,06.2002,12.2014,5,SUV,1


In [226]:
all_models_final['start_production'] = pd.to_datetime(all_models_final['start_production'], format="%m.%Y")
all_models_final['end_production'] = pd.to_datetime(all_models_final['end_production'], format="%m.%Y")

In [227]:

# I cannot find a database that actually has all the information, need to combine multiple data sources
# data is going to be dumped into Excel and edited from there while we collect scraping data. 
all_models_final.to_csv('modelcodes.csv', index = False)